<a href="https://colab.research.google.com/github/ankritRisal/Finetuning_LLM/blob/main/Train_finetunned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Play with data



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [78]:
df = pd.read_csv("/content/drive/MyDrive/X3s4c5/FinetunningLLMmodels/book_description.csv")

In [79]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Title,Category,Price,Avilability,Book_Description,Stars
0,0,0,A Light in the Attic,Poetry,51.77,22,It's hard to imagine a world without A Light i...,3
1,1,1,Tipping the Velvet,Historical Fiction,53.74,20,"""Erotic and absorbing...Written with starling ...",1
2,2,2,Soumission,Fiction,50.10,20,"Dans une France assez proche de la nÃ´tre, un ...",1
3,3,3,Sharp Objects,Mystery,47.82,20,"WICKED above her hipbone, GIRL across her hear...",4
4,4,4,Sapiens: A Brief History of Humankind,History,54.23,20,From a renowned historian comes a groundbreaki...,5


In [80]:
df.drop(["Unnamed: 0", "Unnamed: 0.1", "Price", "Avilability", "Stars", ], axis = 1, inplace = True)

In [81]:
df.head(5)

,Title,Category,Book_Description
0,A Light in the Attic,Poetry,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,Historical Fiction,"""Erotic and absorbing...Written with starling ..."
2,Soumission,Fiction,"Dans une France assez proche de la nÃ´tre, un ..."
3,Sharp Objects,Mystery,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,History,From a renowned historian comes a groundbreaki...


In [82]:
random_seed = 32
df = df.sample(frac=1, random_state = random_seed).reset_index(drop=True).reset_index()
train_size = 0.8
train_len = int(train_size * len(df))

In [83]:
df_train= df[:train_len]
df_test = df[train_len:]

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from torch.utils.data import Dataset, DataLoader

from google.colab import userdata
from huggingface_hub import login

# Get the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

login(token=hf_token)


In [41]:
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-1B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side ="left")
tokenizer.pad_token = tokenizer.eos_token # padding tokens to make of same shape

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             dtype = torch.bfloat16,
                                             device_map = device)

Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

In [42]:
valid_category = list(df["Category"].unique())
system_prompt =  \
  {
    "role" : "system",
    "content" : f""" You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must
    choose from the following classes:
    {"\n or ". join(["Labeled Category:" + x for x in list(valid_category)])}.
    Ensure Output is from above list only"""
    }

post_messages = \
  {
    "role" : "assistant",
    "content" : "Labeled Category :"
    }

# user_messages = \
#  {
#       "role" : "user",
#       "content" : """ Title :A Light in the Attic
#                     Book Description: It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby,
#                     I think someone down here'sGot it in for you. Shel, you never sounded so good. ...more"""
#     }
prompt = [
    system_prompt, post_messages
  ]

In [43]:
chat_template = tokenizer.apply_chat_template(prompt, continue_final_message= True, tokenize= False)
print(chat_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Feb 2026

You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must
    choose from the following classes:
    Labeled Category:Sequential Art
 or Labeled Category:Classics
 or Labeled Category:Humor
 or Labeled Category:Biography
 or Labeled Category:Horror
 or Labeled Category:Young Adult
 or Labeled Category:Romance
 or Labeled Category:Autobiography
 or Labeled Category:Fiction
 or Labeled Category:Business
 or Labeled Category:Default
 or Labeled Category:Psychology
 or Labeled Category:Nonfiction
 or Labeled Category:New Adult
 or Labeled Category:Mystery
 or Labeled Category:Poetry
 or Labeled Category:Historical Fiction
 or Labeled Category:History
 or Labeled Category:Fantasy
 or Labeled Category:Add a comment
 or Labeled Category:Food and Drink
 or Labeled Category:Thriller
 or Labeled Category:Religion
 or

In [13]:
answer = "Hello" # THIS SHOULD BE THE CATEGORY EXTRACTED FROM THE BATCH_LOADER

In [14]:
full_response = chat_template + " " + answer + tokenizer.eos_token
# print(full_response)

In [15]:
tokenized = tokenizer(full_response, return_tensor ="pt", add_special_tokens= False)["input_ids"]
print(tokenized)

[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 2371, 13806, 220, 2366, 21, 271, 2675, 527, 459, 15592, 1887, 430, 16181, 459, 11106, 323, 6017, 7817, 323, 538, 9803, 5699, 315, 279, 2363, 9435, 11, 499, 2011, 198, 262, 5268, 505, 279, 2768, 6989, 512, 262, 445, 23121, 10260, 25, 22821, 5277, 198, 477, 445, 23121, 10260, 25, 1999, 1233, 198, 477, 445, 23121, 10260, 25, 49619, 269, 198, 477, 445, 23121, 10260, 25, 37196, 5814, 198, 477, 445, 23121, 10260, 25, 40701, 7787, 198, 477, 445, 23121, 10260, 25, 41672, 22919, 198, 477, 445, 23121, 10260, 25, 45459, 685, 198, 477, 445, 23121, 10260, 25, 20175, 18843, 5814, 198, 477, 445, 23121, 10260, 70956, 2538, 198, 477, 445, 23121, 10260, 25, 23562, 198, 477, 445, 23121, 10260, 25, 3760, 198, 477, 445, 23121, 10260, 25, 69803, 2508, 198, 477, 445, 23121, 10260, 25, 8284, 58162, 198, 477, 445, 23121, 10260, 25, 3648, 22919, 198, 477, 445, 23121, 10260, 66307, 96357, 198, 477, 4

In [12]:
def extract_labels_from_output(decode_batch, tokenizer):
  # labels = []
  # for d in decode_batch:
  #   if "Labeled Category:" in d:
  #       label = d.split("Labeled Category:")[1].split("<|eot_id|>")[0].strip()
  #       return labels
  #   else:
  #       label = None
  #   labels.append(label)
  labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[1].split("<|eot_id|>")[0].strip() for d in decode_batch]
  return labels

In [13]:
def generate_outputs(prompts, model, tokenizer):
  tokenizer.pad_token = tokenizer.eos_token
  tokenized = tokenizer(prompts, padding= True, return_tensors ="pt", add_special_tokens= False).to(device)

  # tokenized = tokenizer.apply_chat_template(prompts, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
  output_batch = model.generate(input_ids = tokenized["input_ids"], attention_mask =tokenized["attention_mask"], max_new_tokens = 20, do_sample= False, temperature = 0, top_p =1)
  # output_batch = model.generate(tokenized, max_new_tokens = 20)
  decode_batch = tokenizer.batch_decode(output_batch, skip_special_tokens= True)
  prediction = extract_labels_from_output(decode_batch, tokenizer)
  return prediction

In [ ]:

def extract_labels_from_output(decode_batch, tokenizer):
  # labels = []
  # for d in decode_batch:
  #   if "Labeled Category:" in d:
  #       label = d.split("Labeled Category:")[1].split("<|eot_id|>")[0].strip()
  #       return labels
  #   else:
  #       label = None
  #   labels.append(label)
  labels = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  print(labels)
  # return labels

In [ ]:
def test_model(dataloader, model, tokenizer):
  comparison_df= {
      "predictions": [],
      "labels": []
  }

  for batch in dataloader:
    predictions = generate_outputs(prompts= batch["prompt"], model = model, tokenizer= tokenizer) # prompts = batch["prompt"]
    comparison_df["labels"].extend(batch["Category"])
    comparison_df["predictions"].extend(predictions)

  # comparison_df = pd.DataFrame(comparison_df)
  # accuracy = (comparison_df["labels"] == comparison_df["predictions"]).mean()
  # num_invalid_pred = (~comparison_df["predictions"].isin(valid_category)).mean()
  # print(comparison_df.head(10))
  # return {"accuracy ": accuracy,"invalid_predictions": num_invalid_pred}
  return comparison_df

In [ ]:
# WHAT DO BATCH PROMPT SHOULD INCLUDE ? => CHAT_TEMPLATE + " " + CATEGORY_TEMPLATE
# CATEGORY TEMPLATE => TITLE AND DESCRIPTION WITH VALID CATEGORY AS ANSWER (SHOULD OPERATE IN LOOP)
#   SYSTEM PROMPT , PORT PROMPT > CATEGORY TEMPLATE

In [84]:
df.head(1)

,index,Title,Category,Book_Description
0,0,"Death Note, Vol. 5: Whiteout (Death Note #5)",Sequential Art,Light Yagami is an ace student with great pros...


In [174]:
def build_prompt(row):
    valid_category = list(df["Category"].unique())
    SYSTEM_PROMPT =  \
      {
        "role" : "system",
        "content" : f""" You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must
        choose from the following classes:
        {"\n or ". join(["Labeled Category:" + x for x in list(valid_category)])}.
        Ensure Output is from above list only"""
        }

    ASSISTANT_MESSAGES = \
      {
        "role" : "assistant",
        "content" : "Labeled Category :"
        }

    USER_MESSAGES = {
        "role": "user",
        "content": f"""Title: {row['Title']}
Description: {row['Book_Description']}"""
    }

    prompt = [
        SYSTEM_PROMPT,
        USER_MESSAGES,
        ASSISTANT_MESSAGES
    ]
    tokenized_prompt = tokenizer.apply_chat_template(prompt, continue_final_message= True, tokenize= False)
    return tokenized_prompt

In [175]:
df["prompt"] = df.apply(build_prompt, axis=1)

In [176]:
def get_dataset(tokenizer, train_size):
  df = pd.read_csv("/content/drive/MyDrive/X3s4c5/FinetunningLLMmodels/book_description.csv")

  df.drop(["Unnamed: 0", "Unnamed: 0.1", "Price", "Avilability", "Stars", ], axis = 1, inplace = True)

  df["prompt"] = df.apply(build_prompt, axis=1)
  df = df.sample(frac=1, random_state = random_seed).reset_index(drop=True).reset_index()
  train_len = int(train_size * len(df))
  df_train= df[:train_len]
  df_test = df[train_len:]

  return df_train, df_test

In [177]:
def extract_labels_from_output(decode_batch, tokenizer):
  labels = []
  for d in decode_batch:
    # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # print(d)
    label = d.split("Labeled Category :")[1].strip()
    # print(label)
    labels.append(label)
  # labels = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  # print(labels)
  # labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[1].split("<|eot_id|>")[0].strip() for d in decode_batch]
  return labels

In [178]:
def generate_outputs(prompts, model, tokenizer):
  tokenizer.pad_token = tokenizer.eos_token
  tokenized = tokenizer(prompts, padding= True, return_tensors ="pt", add_special_tokens= False).to(device)

  # tokenized = tokenizer.apply_chat_template(prompts, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
  output_batch = model.generate(input_ids = tokenized["input_ids"], attention_mask =tokenized["attention_mask"], max_new_tokens = 20, do_sample= False, temperature = 0, top_p =1)
  # output_batch = model.generate(tokenized, max_new_tokens = 20)
  decode_batch = tokenizer.batch_decode(output_batch, skip_special_tokens= True)
  prediction = extract_labels_from_output(decode_batch, tokenizer)
  return prediction

In [179]:
def test_model(dataloader, model, tokenizer):
  comparison_df= {
      "predictions": [],
      "labels": []
  }

  for batch in dataloader:
    predictions = generate_outputs(prompts= batch["prompt"], model = model, tokenizer= tokenizer) # prompts = batch["prompt"]
    comparison_df["labels"].extend(batch["Category"])
    comparison_df["predictions"].extend(predictions)

  # comparison_df = pd.DataFrame(comparison_df)
  # accuracy = (comparison_df["labels"] == comparison_df["predictions"]).mean()
  # num_invalid_pred = (~comparison_df["predictions"].isin(valid_category)).mean()
  # print(comparison_df.head(10))
  # return {"accuracy ": accuracy,"invalid_predictions": num_invalid_pred}
  return comparison_df

In [180]:
from datasets import Dataset #defined by huggingface not pandas or tf
batch_size = 16
train_df, test_df = get_dataset(tokenizer = tokenizer, train_size = 0.95)
test_Dataset = Dataset.from_pandas(df_test)
test_dataloader = DataLoader(test_Dataset, batch_size = batch_size, shuffle= False)
metrics = test_model(test_dataloader, model, tokenizer)
# print("\n".join([f"{k} = {v}" for k, v in metrics.items()]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [184]:
comparison = pd.DataFrame(metrics)
accuracy = (comparison["labels"] == comparison["predictions"]).mean()
num_invalid_pred = (~comparison["predictions"].isin(valid_category)).mean()
print(comparison.head(10))
print(f"accuracy : {accuracy}, invalid_predictions: {num_invalid_pred}")

   predictions             labels
0  Young Adult     Sequential Art
1       Memoir         Psychology
2       Horror            Mystery
3       Horror        Young Adult
4  Young Adult            Fantasy
5      Romance            Romance
6  Young Adult  Christian Fiction
7  Young Adult            Fiction
8      Science         Nonfiction
9  Young Adult            Mystery
accuracy : 0.185, invalid_predictions: 0.09


In [194]:
title_name = "Ouran High School Host Club, Vol. 1 (Ouran High School Host Club #1)"
df[df["Title"] == title_name]

,level_0,index,Title,Category,Book_Description,prompt
800,800,470,"Ouran High School Host Club, Vol. 1 (Ouran Hig...",Sequential Art,"One day, Haruhi, a scholarship student at excl...",<|begin_of_text|><|start_header_id|>system<|en...


dataloader is object but we need list

In [124]:
for batch in test_dataloader:
  print(batch["prompt"])

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 Feb 2026\n\nYou are an AI system that reads an Title and Book Description and classifies category of the book applied, you must\n        choose from the following classes:\n        Labeled Category:Sequential Art\n or Labeled Category:Classics\n or Labeled Category:Humor\n or Labeled Category:Biography\n or Labeled Category:Horror\n or Labeled Category:Young Adult\n or Labeled Category:Romance\n or Labeled Category:Autobiography\n or Labeled Category:Fiction\n or Labeled Category:Business\n or Labeled Category:Default\n or Labeled Category:Psychology\n or Labeled Category:Nonfiction\n or Labeled Category:New Adult\n or Labeled Category:Mystery\n or Labeled Category:Poetry\n or Labeled Category:Historical Fiction\n or Labeled Category:History\n or Labeled Category:Fantasy\n or Labeled Category:Add a comment\n or Labeled Category:Food and Drink\n or Labeled Category:Thri